# Tektronix Keithley 2182A Nanovoltmeter Driver Example

This notebook demonstrates the usage of the QCoDeS driver for the Keithley 2182A nanovoltmeter. The Keithley 2182A is a precision nanovoltmeter capable of measuring voltages down to the nanovolt level with exceptional accuracy.

## Key Features

- **High-precision voltage measurements** down to nanovolt levels
- **Temperature measurements** with multiple unit support
- **MEASure and FETCh commands** for optimal measurement workflows
- **Comprehensive noise reduction** features
- **Flexible trigger system** for precise measurement timing
- **Statistical analysis** capabilities

In [ ]:
# Import necessary modules
import numpy as np
import matplotlib.pyplot as plt
from qcodes_contrib_drivers.drivers.Tektronix.Keithley_2182A import Keithley2182A

# For demonstration purposes, we'll use a simulated instrument
# In real usage, replace with your actual VISA address
import qcodes.instrument.sims as sims
visalib = sims.__file__.replace('__init__.py', 'Keithley_2182A.yaml@sim')

## 1. Instrument Initialization and Basic Setup

In [ ]:
# Initialize the Keithley 2182A
# For real hardware, use something like: 'TCPIP::192.168.1.100::INSTR'
voltmeter = Keithley2182A(
    'k2182a', 
    'TCPIP::192.168.1.100::INSTR',
    visalib=visalib  # Remove this line for real hardware
)

# Check instrument identification
print(f"Connected to: {voltmeter.get_idn()}")

## 2. Basic Voltage Measurements

The Keithley 2182A excels at high-precision voltage measurements. Let's start with basic configuration and measurements.

In [ ]:
# Configure for voltage measurements
voltmeter.configure_voltage_measurement(
    auto_range=True,
    nplc=1.0,        # 1 power line cycle for balanced speed/accuracy
    auto_zero=True   # Enable auto-zero for best accuracy
)

print(f"Measurement mode: {voltmeter.mode()}")
print(f"Auto-range enabled: {voltmeter.auto_range_enabled()}")
print(f"Integration time (NPLC): {voltmeter.nplc()}")
print(f"Auto-zero enabled: {voltmeter.get_auto_zero()}")

In [ ]:
# Take a single voltage measurement
voltage = voltmeter.voltage()
print(f"Measured voltage: {voltage:.6f} V")
print(f"Measured voltage: {voltage*1e6:.3f} µV")
print(f"Measured voltage: {voltage*1e9:.1f} nV")

## 3. MEASure vs FETCh Commands

The Keithley 2182A supports both MEASure and FETCh commands:
- **MEASure**: Triggers a new measurement and returns the result
- **FETCh**: Retrieves the last measurement from the buffer without triggering

In [ ]:
# MEASure command - triggers new measurement
measured_value = voltmeter._measure_voltage()
print(f"MEASure command result: {measured_value:.6f} V")

# FETCh command - gets buffered result (should be the same)
fetched_value = voltmeter.fetch()
print(f"FETCh command result: {fetched_value:.6f} V")

# READ command - combines trigger and fetch
read_value = voltmeter.read()
print(f"READ command result: {read_value:.6f} V")

## 4. Optimization for Different Measurement Requirements

The driver provides preset configurations for different measurement scenarios.

In [ ]:
# Optimize for maximum accuracy (low noise)
print("=== Low Noise Configuration ===")
voltmeter.optimize_for_low_noise()

print(f"NPLC: {voltmeter.nplc()}")
print(f"Averaging enabled: {voltmeter.averaging_enabled()}")
print(f"Averaging count: {voltmeter.averaging_count()}")
print(f"Analog filter: {voltmeter.analog_filter()}")
print(f"Digital filter: {voltmeter.digital_filter()}")

# Take a measurement with low noise settings
low_noise_voltage = voltmeter.voltage()
print(f"Low noise measurement: {low_noise_voltage:.6f} V")

In [ ]:
# Optimize for speed
print("=== Fast Measurement Configuration ===")
voltmeter.optimize_for_speed()

print(f"NPLC: {voltmeter.nplc()}")
print(f"Averaging enabled: {voltmeter.averaging_enabled()}")
print(f"Analog filter: {voltmeter.analog_filter()}")
print(f"Digital filter: {voltmeter.digital_filter()}")

# Take a fast measurement
fast_voltage = voltmeter.voltage()
print(f"Fast measurement: {fast_voltage:.6f} V")

## 5. Statistical Analysis of Measurements

For characterizing noise and stability, the driver provides built-in statistical analysis.

In [ ]:
# Configure for medium accuracy/speed balance
voltmeter.set_measurement_speed('medium')

# Take multiple measurements for statistical analysis
num_measurements = 20
stats = voltmeter.measure_voltage_statistics(num_measurements=num_measurements)

print(f"Statistical Analysis (n={stats['count']})")
print(f"Mean voltage: {stats['mean']:.6f} V")
print(f"Standard deviation: {stats['stdev']:.9f} V ({stats['stdev']*1e9:.2f} nV)")
print(f"Minimum: {stats['min']:.6f} V")
print(f"Maximum: {stats['max']:.6f} V")
print(f"Peak-to-peak: {(stats['max'] - stats['min']):.9f} V ({(stats['max'] - stats['min'])*1e9:.2f} nV)")

In [ ]:
# Plot the measurement data
measurements = stats['measurements']
measurement_numbers = range(1, len(measurements) + 1)

plt.figure(figsize=(12, 8))

# Time series plot
plt.subplot(2, 2, 1)
plt.plot(measurement_numbers, np.array(measurements)*1e6, 'b.-', markersize=8)
plt.axhline(y=stats['mean']*1e6, color='r', linestyle='--', label=f'Mean: {stats["mean"]*1e6:.3f} µV')
plt.xlabel('Measurement Number')
plt.ylabel('Voltage (µV)')
plt.title('Voltage Measurements vs Time')
plt.legend()
plt.grid(True, alpha=0.3)

# Histogram
plt.subplot(2, 2, 2)
plt.hist(np.array(measurements)*1e6, bins=10, alpha=0.7, edgecolor='black')
plt.xlabel('Voltage (µV)')
plt.ylabel('Frequency')
plt.title('Voltage Distribution')
plt.grid(True, alpha=0.3)

# Deviation from mean
plt.subplot(2, 2, 3)
deviations = (np.array(measurements) - stats['mean']) * 1e9
plt.plot(measurement_numbers, deviations, 'g.-', markersize=8)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Measurement Number')
plt.ylabel('Deviation from Mean (nV)')
plt.title('Measurement Stability')
plt.grid(True, alpha=0.3)

# Box plot
plt.subplot(2, 2, 4)
plt.boxplot(np.array(measurements)*1e6, patch_artist=True)
plt.ylabel('Voltage (µV)')
plt.title('Voltage Statistics')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Temperature Measurements

The Keithley 2182A can also measure temperature using appropriate probes.

In [ ]:
# Configure for temperature measurement
voltmeter.configure_temperature_measurement(
    units='celsius',
    nplc=1.0
)

print(f"Measurement mode: {voltmeter.mode()}")
print(f"Temperature units: {voltmeter.temperature_units()}")

# Measure temperature (requires appropriate probe)
try:
    temperature_c = voltmeter.temperature()
    print(f"Temperature: {temperature_c:.2f} °C")
    
    # Convert to other units
    temperature_k = temperature_c + 273.15
    temperature_f = temperature_c * 9/5 + 32
    
    print(f"Temperature: {temperature_k:.2f} K")
    print(f"Temperature: {temperature_f:.2f} °F")
    
except Exception as e:
    print(f"Temperature measurement note: {e}")
    print("(Temperature measurement requires appropriate probe)")

## 7. Advanced Trigger System

The instrument supports various trigger sources for precise measurement timing.

In [ ]:
# Set back to voltage mode
voltmeter.mode('dc voltage')

# Configure trigger system
print("=== Trigger System Configuration ===")

# Set trigger source to manual
voltmeter.trigger_source('manual')
print(f"Trigger source: {voltmeter.trigger_source()}")

# Set trigger delay
voltmeter.trigger_delay(0.1)
print(f"Trigger delay: {voltmeter.trigger_delay()} s")

# Initiate measurement (puts instrument in wait-for-trigger state)
voltmeter.initiate_measurement()
print("Measurement initiated - waiting for trigger...")

# Send software trigger
voltmeter.trigger()
print("Trigger sent")

# Fetch the result
triggered_result = voltmeter.fetch()
print(f"Triggered measurement result: {triggered_result:.6f} V")

# Reset to immediate triggering
voltmeter.trigger_source('immediate')
print(f"Reset trigger source to: {voltmeter.trigger_source()}")

## 8. Comprehensive Status Monitoring

The driver provides detailed status information for monitoring instrument configuration.

In [ ]:
# Get comprehensive measurement status
status = voltmeter.get_measurement_status()

print("=== Instrument Status ===")
for key, value in status.items():
    print(f"{key:20}: {value}")

# Check available ranges
print("\n=== Range Information ===")
ranges = voltmeter.check_ranges()
for key, value in ranges.items():
    print(f"{key:20}: {value}")

## 9. Error Handling and Diagnostics

In [ ]:
# Check for errors
error_code, error_message = voltmeter.get_error()
print(f"Error status: {error_code} - {error_message}")

# Perform self-test
self_test_result = voltmeter.self_test()
print(f"Self-test result: {'PASSED' if self_test_result else 'FAILED'}")

# Clear any errors
voltmeter.clear_errors()
print("Error queue cleared")

## 10. Measurement Speed Comparison

Let's compare the three measurement speed presets.

In [ ]:
import time

speed_settings = ['fast', 'medium', 'slow']
measurement_times = []
measurement_results = []

for speed in speed_settings:
    print(f"\n=== Testing {speed.upper()} speed setting ===")
    
    # Set speed
    voltmeter.set_measurement_speed(speed)
    
    # Display current settings
    print(f"NPLC: {voltmeter.nplc()}")
    print(f"Analog filter: {voltmeter.analog_filter()}")
    print(f"Digital filter: {voltmeter.digital_filter()}")
    
    # Time a series of measurements
    num_measurements = 10
    start_time = time.time()
    
    measurements = []
    for i in range(num_measurements):
        measurements.append(voltmeter.voltage())
    
    end_time = time.time()
    total_time = end_time - start_time
    
    measurement_times.append(total_time)
    measurement_results.append(measurements)
    
    print(f"Time for {num_measurements} measurements: {total_time:.3f} s")
    print(f"Average time per measurement: {total_time/num_measurements:.3f} s")
    print(f"Mean voltage: {np.mean(measurements):.6f} V")
    print(f"Std deviation: {np.std(measurements):.9f} V ({np.std(measurements)*1e9:.2f} nV)")

In [ ]:
# Plot speed comparison
plt.figure(figsize=(15, 5))

# Measurement time comparison
plt.subplot(1, 3, 1)
plt.bar(speed_settings, measurement_times, color=['red', 'orange', 'blue'], alpha=0.7)
plt.xlabel('Speed Setting')
plt.ylabel('Time for 10 measurements (s)')
plt.title('Measurement Speed Comparison')
plt.grid(True, alpha=0.3)

# Noise comparison (standard deviation)
plt.subplot(1, 3, 2)
noise_levels = [np.std(results)*1e9 for results in measurement_results]
plt.bar(speed_settings, noise_levels, color=['red', 'orange', 'blue'], alpha=0.7)
plt.xlabel('Speed Setting')
plt.ylabel('Measurement Noise (nV RMS)')
plt.title('Noise Level Comparison')
plt.grid(True, alpha=0.3)

# Noise vs Speed trade-off
plt.subplot(1, 3, 3)
colors = ['red', 'orange', 'blue']
for i, (speed, time_per_meas, noise) in enumerate(zip(speed_settings, 
                                                     [t/10 for t in measurement_times], 
                                                     noise_levels)):
    plt.scatter(time_per_meas, noise, s=100, c=colors[i], alpha=0.7, label=speed)
    plt.annotate(speed, (time_per_meas, noise), xytext=(5, 5), 
                textcoords='offset points', fontsize=10)

plt.xlabel('Time per Measurement (s)')
plt.ylabel('Measurement Noise (nV RMS)')
plt.title('Speed vs Noise Trade-off')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Cleanup and Best Practices

In [ ]:
# Final status check
print("=== Final Status Check ===")
final_status = voltmeter.get_measurement_status()
print(f"Current mode: {final_status['mode']}")
print(f"Current range: {final_status['range']} V")
print(f"Current NPLC: {final_status['nplc']}")

# Check for any errors before closing
error_code, error_message = voltmeter.get_error()
if error_code == 0:
    print("No errors detected")
else:
    print(f"Warning: Error {error_code} - {error_message}")

# Close the instrument connection
voltmeter.close()
print("\nInstrument connection closed.")

## Summary

This notebook demonstrated the key features of the Keithley 2182A driver:

1. **Basic Setup**: Instrument initialization and identification
2. **Voltage Measurements**: High-precision nanovolt measurements
3. **MEASure vs FETCh**: Understanding different measurement commands
4. **Optimization**: Presets for different accuracy/speed requirements
5. **Statistical Analysis**: Built-in measurement statistics and visualization
6. **Temperature Measurements**: Temperature probe support
7. **Trigger System**: Advanced triggering for precise timing
8. **Status Monitoring**: Comprehensive instrument status
9. **Error Handling**: Diagnostics and error management
10. **Performance Analysis**: Speed vs noise trade-offs

### Key Takeaways:

- **Fast measurements** (0.1 NPLC): Best for speed, higher noise
- **Medium measurements** (1.0 NPLC): Balanced speed and accuracy
- **Slow measurements** (10 NPLC): Maximum accuracy, lower speed
- **Filters and averaging** significantly improve measurement stability
- **Auto-zero** is essential for best accuracy in DC measurements
- **Statistical analysis** helps characterize measurement uncertainty

### For Real Hardware Usage:

1. Remove the `visalib=visalib` parameter when connecting to real hardware
2. Use appropriate VISA addresses (e.g., `'TCPIP::192.168.1.100::INSTR'`)
3. Ensure proper grounding and shielding for nanovolt measurements
4. Allow adequate warm-up time for maximum accuracy
5. Consider environmental factors (temperature, vibration, EMI)

The Keithley 2182A driver provides a comprehensive interface for precise voltage and temperature measurements in scientific and industrial applications.